## U.S. Bureau of Labor Statistics - CPI Analysis
#### Eric Bottinelli

### 1. Retrieve data via BLS API v2

Doc:
- https://www.bls.gov/developers/api_python.htm
- https://data.bls.gov/cgi-bin/surveymost?bls

Packages to install:
- Prettytable ('pip install prettytable')

In [5]:
import os
import requests
import json
import prettytable
from datetime import datetime

folder_name = 'CPI_Data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

current_date = datetime.now()
current_year = current_date.year
last_year = current_year - 1

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','CUUR0000AA0'],"startyear":str(last_year), "endyear":str(current_year)})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    file_path = os.path.join(folder_name, seriesId + '.txt')
    with open(file_path, 'w') as output:
        output.write(x.get_string())
    output.close()

In [3]:
output

<_io.TextIOWrapper name='CUUR0000AA0.txt' mode='w' encoding='UTF-8'>